# ¿Es confiable la información de GeoGPS Perú?

#### Introducción
Hago este cuaderno porque aunque GeoGPS Perú me brinde información de todas las manzanas del país no es una fuente oficial.

INEI sí lo es, pero como podrán apreciar más adelante, no brinda datos sobre población para todas las manzanas de las *ciudades principales*, sino solo para un 56% de estas. 

Entonces, mi intención es comparar los datos de este 56% por ciento que INEI me brinda con los que GeoGPS tiene. Lo que espero es que la información de población para cada una de estas manzanas coincida entre INEI y GeoGPS Perú.

#### Metodología
Lo primero que haremos será importar el gdf que INEI me envió por correo para recalcular sus IDs de Manzanas.

Por defecto en este gdf hay un ID de Manzana conformado por *UBIGEO + CODZONA + SUFZONA + CODMZNA SUFMZNA + COD_MANZANA*, pero cuando uno descarga datos de INEI a nivel manzana, los ID de Manzanas tienen la siguiente forma: *UBIGEO + **CODCCPP** + CODZONA + SUFZONA + CODMZNA SUFMZNA + COD_MANZANA*.

Por tanto, recalcularemos ese ID de Manzana, ya que el gdf como podrán ver tiene info por separado de cada uno de los componentes. 

Una vez hecho eso, se hará un match entre este gdf con la información descargada directamente desde INEI. Luego, se podrá comparar con lo que GeoGPS brinda.

In [ ]:
import os
oschdir("G:/Mi unidad/Documentos personales/Muertes Covid-19 por Áreas urbanas - Perú/1-Cálculos previos con límites INEI")
import pandas as pd
import geopandas as gpd
import numpy as np

#1.Lo único que hare en esta parte será importar el archivo y añadirle un ID a cada manzana.
#El resultado de esto lo trabajaré en R para obtener un df que tenga ID e ID-Manzana corregida que luego
#se pueda juntar a este original.
gdf=gpd.read_file("rawdata/Mz_PrincipalesCiudades.shp") #archivo original enviado por INEI
gdf.head()

In [ ]:
gdf['ID']=gdf.index #le añado un ID para luego poder hacer el match
gdf.head()

In [ ]:
#2.Exporto para trabajar en R.
gdf.to_csv("P1-INEI vs GeoGPS-Peru/P1-data/1-Mz_PrincipalesCiudades.csv") #exporto para poder trabajar en R

In [ ]:
#3.Después de haberlo trabajado en R('P1-1-Corrigiendo ID de manzanas.R'), importo el df que quería.
ids=pd.read_csv("P1-INEI vs GeoGPS-Peru/P1-data/2-CodigoManzanasOficial.csv")
ids=ids.drop("Unnamed: 0",axis=1)
ids.head()

In [ ]:
#4. Hago el match
gdf_nuevo = gpd.GeoDataFrame(gdf.merge(ids, on='ID', how='left')) 
gdf_nuevo.head()
#Como pueden ver, lo único que obtengo es el gdf original con un ID de Manzana que ahora sí podrá
#coincidir con el que INEI te da cuando descargas datos del Censo 2017.

In [ ]:
#5.Importo el df con datos de población a nivel de manzana descargado de INEI
#(Pre-procesado en 'P1-2-Pob x Manzana INEI.R')
mzpob=pd.read_csv("P1-INEI vs GeoGPS-Peru/P1-data/3-PobMz_INEI_OK.csv") #Este es el archivo que obtengo luego de procesar el original
                                              #en R
mzpob=mzpob.drop("Unnamed: 0",axis=1)
mzpob.rename(columns={'Poblaci�n':'Poblacion'}, 
               inplace=True)
gdf_final = gpd.GeoDataFrame(gdf_nuevo.merge(mzpob, on='Mz', how='left'))
gdf_final=gdf_final[['ID','Mz','Poblacion']]
gdf_final.head()

In [ ]:
#6.Exporto para poder comparar luego con el de GeoGPS Perú
gdf_final.to_csv("P1-INEI vs GeoGPS-Peru/P1-data/4-Mzs-Poblacion-Oficial-INEI.csv")
gdf_final.to_file("P1-INEI vs GeoGPS-Peru/P1-data/4-Mzs-Poblacion-Oficial-INEI.geojson",
                  driver='GeoJSON') #por si quiero revisar manualmente en QGIS

len(gdf_final) #174362

In [ ]:
#7.Ahora, los datos de GeoGPS Perú están en un gdf. Así que primero lo importo
gdf2=gpd.read_file('rawdata/PeruEnManzanas_INEI2017_GC_18S.gpkg')
#8.Y ahora lo exporto para poder compararlo como csv en R.
gdf2.to_csv("P1-INEI vs GeoGPS-Peru/P1-data/5-PobJuanSuyo.csv")

In [ ]:
#9.El análisis final se encuentra en 'P1-3-Comparacion GEOGPS vs INEI.R'
#El resultado es que de 98486 manzanas de las que se tiene informacion oficial de INEI, 984392 tienen la 
#misma informacion en GeoGPS-Peru. Esto representa el 99.99045%.